Initially this notebook was created on a different machine on the lan, however the api was not accessible.
So the notebook has to be run on the server, in which the gradio app is running

In [1]:
import gradio as gr
from gradio_client import Client

In [2]:
import torch
import transformers
from transformers import AutoTokenizer

In [3]:
tokenizer = AutoTokenizer.from_pretrained("codellama/CodeLlama-7b-hf")

pipeline = transformers.pipeline(
    "text-generation",
    model="codellama/CodeLlama-7b-hf",
    torch_dtype=torch.float16,
    device_map="auto",
)

In [4]:
from transformers import BitsAndBytesConfig

nf4_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type="nf4",
   bnb_4bit_use_double_quant=True,
   bnb_4bit_compute_dtype=torch.bfloat16
)

In [5]:
from transformers import AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained("codellama/CodeLlama-7b-hf", quantization_config=nf4_config)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
text = "def txt_to_json("
device = "cuda:0"

inputs = tokenizer(text, return_tensors="pt").to(device)
outputs = model.generate(**inputs, 
                         max_new_tokens=200)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [7]:
print(tokenizer.decode(outputs[0], 
                       skip_special_tokens=True))

def txt_to_json(txt_file):
    with open(txt_file, 'r') as f:
        lines = f.readlines()
        json_lines = []
        for line in lines:
            json_lines.append(json.loads(line))
        return json_lines


def json_to_txt(json_file, txt_file):
    with open(json_file, 'r') as f:
        lines = f.readlines()
        txt_lines = []
        for line in lines:
            txt_lines.append(json.dumps(line))
        with open(txt_file, 'w') as f:
            for line in txt_lines:
                f.write(line)
                f.write('\n')


def txt_to_csv(txt_file, csv_file):
    with open(txt_file,


In [6]:
import locale

def getpreferredencoding(do_setlocale=True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [7]:
def predict(prompt):
    device = "cuda:0"
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    outputs = model.generate(**inputs, 
                         max_new_tokens=200)
    generated_text = tokenizer.decode(outputs[0], 
                                      skip_special_tokens=True)
    return generated_text

In [8]:
demo = gr.Interface(
  fn=predict,
  inputs=gr.Textbox(label="Please, write your request here:", placeholder="example: def fibonacci(", lines=5),
  outputs=gr.Textbox(label="Answer (inference):"),
  title='On Premise Code LLama2 Helper',
  description='description',
  article='My article on Medium https://medium.com',
  examples=[["def Fibonacci("], ["function DotProduct("], ['springboot profile'], ['write a class for manage shipment']],
  allow_flagging="never"
)

In [9]:
demo.launch(share=True)

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://1dab380cb604a34025.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


In [16]:
demo.close()

Closing server running on port: 7860


In [10]:
from gradio_client import Client

test_client = Client(src= "http://127.0.0.1:7860")

Loaded as API: http://127.0.0.1:7860/ ✔


In [11]:
test_client.view_api(all_endpoints=True)

Client.predict() Usage Info
---------------------------
Named API endpoints: 1

 - predict(please_write_your_request_here, api_name="/predict") -> answer_inference
    Parameters:
     - [Textbox] please_write_your_request_here: str 
    Returns:
     - [Textbox] answer_inference: str 

Unnamed API endpoints: 0



In [16]:
test_client.predict('count possum', api_name='/predict')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


'count possum\n  end\n\n  def test_possum_with_possum_and_possum\n    assert_equal 3, possum possum possum\n  end\n\n  def test_possum_with_possum_and_possum_and_possum\n    assert_equal 4, possum possum possum possum\n  end\n\n  def test_possum_with_possum_and_possum_and_possum_and_possum\n    assert_equal 5, possum possum possum possum possum\n  end\n\n  def test_possum_with_possum_and_possum_and_possum_and_possum_and_possum\n    assert_equal 6, possum possum possum possum possum possum\n  end\n\n  def test_possum_with_possum_and'